### Initialization and Starter Code

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from datetime import datetime
from tqdm.auto import tqdm #progress bars to see progress 

SCORE_RANGES = {
        1: {'sentence_fluency': (1, 6), 'word_choice': (1, 6), 'conventions': (1, 6),'organization': (1, 6),
            'content': (1, 6), 'holistic': (2, 12)},
        2: {'sentence_fluency': (1, 6), 'word_choice': (1, 6), 'conventions': (1, 6),'organization': (1, 6),
            'content': (1, 6), 'holistic': (1, 6)},
        3: {'narrativity': (0, 3), 'language': (0, 3), 'prompt_adherence': (0, 3), 'content': (0, 3),
            'holistic': (0, 3)},
        4: {'narrativity': (0, 3), 'language': (0, 3), 'prompt_adherence': (0, 3), 'content': (0, 3),
            'holistic': (0, 3)},
        5: {'narrativity': (0, 4), 'language': (0, 4), 'prompt_adherence': (0, 4), 'content': (0, 4),
            'holistic': (0, 4)},
        6: {'narrativity': (0, 4), 'language': (0, 4), 'prompt_adherence': (0, 4), 'content': (0, 4),
            'holistic': (0, 4)},
        7: {'conventions': (0, 6), 'organization': (0, 6), 'content': (0, 6),'holistic': (0, 30)},
        8: {'sentence_fluency': (2, 12), 'word_choice': (2, 12), 'conventions': (2, 12),'organization': (2, 12),
            'content': (2, 12), 'holistic': (0, 60)}}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #use gpu if avilable 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') #initialize BERT tokenizer 
def read_data(path):
   data = pd.read_csv(path)
   return {
       'essay_ids': data['essay_id'].values,
       'prompt_ids': data['prompt_id'].values,
       'essay_text': data['essay_text'].values,
       'features': data.iloc[:, 12:].values,
       'holistic': data['holistic'].values
   }

def normalize_scores(scores, prompt_id): #helper function to normalize scores since we have dif ranges
   score_range = SCORE_RANGES[prompt_id]['holistic']
   return (scores - score_range[0]) / (score_range[1] - score_range[0])

def denormalize_scores(norm_scores, prompt_id):
   score_range = SCORE_RANGES[prompt_id]['holistic']
   return norm_scores * (score_range[1] - score_range[0]) + score_range[0]

def quadratic_weighted_kappa(y_true, y_pred):
   return cohen_kappa_score(y_true, np.round(y_pred), weights='quadratic')

class essays_dataset(Dataset): #class to help organize and prepare our data for training
   def __init__(self, essays, features, scores, prompt_ids, max_length=512):
       self.essays = essays
       self.features = features
       self.scores = scores
       self.prompt_ids = prompt_ids
       self.max_length = max_length #max tokens length for BERT which is 512

   def __len__(self):
       return len(self.essays) #returns number of essays in the dataset

   def __getitem__(self, idx): #to get a single essay with its info 
       essay = str(self.essays[idx])
       features = self.features[idx]
       score = self.scores[idx]
       prompt_id = self.prompt_ids[idx]

        #tokenize the essay text 
       encoding = tokenizer(  
           essay,
           max_length=self.max_length, #BERTs max length 512
           padding='max_length', #add paddinf if <512 to reach 512
           truncation=True, #cut off if >512
           return_tensors='pt'
       )

        #return dict with all needed info
       return {
           'input_ids': encoding['input_ids'].squeeze(), #token id and remove extra single dimension
           'attention_mask': encoding['attention_mask'].squeeze(), #mask to ignore padding 
           'features': torch.FloatTensor(features), 
           'score': torch.FloatTensor([score]),
           'prompt_id': prompt_id
       }

class Our_BERT(nn.Module):
   def __init__(self, hidden_size=0):
       super(Our_BERT, self).__init__()
       #load pretrained bert model
       self.bert = BertModel.from_pretrained('bert-base-uncased')
       
       input_size = 768 + 86 #768 from BERT 86 our features
       
        #create regression head based on hidden_size
       if hidden_size > 0:  #ff hidden size sis pecified use a hidden layer with ReLU
           self.regression_head = nn.Sequential(
               nn.Linear(input_size, hidden_size),
               nn.ReLU(),
               nn.Linear(hidden_size, 1)
           )
       else: #otherwise use single layer
           self.regression_head = nn.Linear(input_size, 1)
        #initialize weights using He/Kaiming initialization
       for module in self.regression_head.modules():
           if isinstance(module, nn.Linear):
               nn.init.kaiming_normal_(module.weight)
               nn.init.zeros_(module.bias)

   def forward(self, input_ids, attention_mask, features): #forward pass
       #create an embedding for each token in our essay text
       outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True) 
       cls_embedding = outputs.last_hidden_state[:, 0, :] #get CLS token embedding which represents essay
       combined_features = torch.cat([cls_embedding, features], dim=1) #combine CLS with features
       return self.regression_head(combined_features) #pass them through the regression head 

def train_model(model, train_loader, criterion, optimizer, epoch):
   model.train() #set the model in training mode 
   total_loss = 0
   batches_no = 0

    #iterate through batches with progress bar
   for batch in tqdm(train_loader, desc=f'Epoch {epoch}', leave=False):
       input_ids = batch['input_ids'].to(device) #move all batch data to correct device so gpu or cpu
       attention_mask = batch['attention_mask'].to(device)
       features = batch['features'].to(device)
       scores = batch['score'].to(device)

       optimizer.zero_grad() #clear gradients from before
       outputs = model(input_ids, attention_mask, features) #forward pass
       loss = criterion(outputs, scores) #calculate loss
       loss.backward() #backprobagation
       optimizer.step() #update weights
        #track the total loss
       total_loss += loss.item()
       batches_no += 1

   avg_loss = total_loss / batches_no #calculate avg loss for each epoch
   print(f'epoch {epoch}: avg training loss = {avg_loss:.4f}')
   return avg_loss

def evaluate_model(model, val_loader, criterion, val_prompt_id):
   model.eval() #set model in evaluation mode
   total_loss = 0
   batches_no = 0
   all_predictions = [] #to store all our predictions
   all_real_scores = [] #to store all the real scores

   print("\n--------------------Evaluating-----------------------")
   with torch.no_grad(): #don't track gradients during evaluation
       for batch in tqdm(val_loader, desc='Validation', leave=False):
           input_ids = batch['input_ids'].to(device)
           attention_mask = batch['attention_mask'].to(device)
           features = batch['features'].to(device)
           scores = batch['score'].to(device)

           outputs = model(input_ids, attention_mask, features) 
           loss = criterion(outputs, scores) #calculate loss

           total_loss += loss.item()
           batches_no += 1
            #convert normalized scores back to original range
           predictions = denormalize_scores(outputs.cpu().numpy(), val_prompt_id)
           targets = denormalize_scores(scores.cpu().numpy(), val_prompt_id)
           all_predictions.extend(predictions)
           all_real_scores.extend(targets)
    #calculate average loss and QWK score
   avg_loss = total_loss / batches_no
   qwk = quadratic_weighted_kappa(np.round(all_real_scores), np.round(all_predictions))
   
   print(f"\nValidation results for prompt {val_prompt_id}:")
   print(f"Avg loss: {avg_loss:.4f}")
   print(f"QWK: {qwk:.4f}")
   #print min, max and mean of predicted and actual to see if we are normalizing and denormalizing properly and how far off we are from actual values
   print(f"Predicted min: {np.min(all_predictions):.2f}, max: {np.max(all_predictions):.2f}, mean: {np.mean(all_predictions):.2f}")
   print(f"Actual min: {np.min(all_real_scores):.2f}, max: {np.max(all_real_scores):.2f}, mean: {np.mean(all_real_scores):.2f}")
   
   return avg_loss, qwk



### Grid search

In [ ]:

print(f"\n{'='*50}")
print(f"Starting Holistic scoring using BERT model at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"{'='*50}")

data_dict = read_data('dataset.csv')
validation_prompts = list(range(2, 9)) #prompts 2-8 used for cross validation
results = [] #to store the results for each time
hidden_sizes = [0, 4, 8]

for hidden_size in hidden_sizes: #grid search over dif regression head configs
    print(f"\n{'='*50}")
    print(f"Trying regression head with {hidden_size} hidden units")
    fold_scores = [] #to store the scores for each validation prompt

    for val_prompt in validation_prompts:
        print(f"\n{'-'*50}")
        print(f"Validation prompt no. {val_prompt}")
        #create train and validation splits
        train_mask = np.isin(data_dict['prompt_ids'], [prompt for prompt in validation_prompts if prompt != val_prompt])
        val_mask = (data_dict['prompt_ids'] == val_prompt)

        #create datasets using the class we defined for training and validation
        train_dataset = essays_dataset(
            data_dict['essay_text'][train_mask],
            data_dict['features'][train_mask],
            [normalize_scores(score, pid) for score, pid in zip(data_dict['holistic'][train_mask], data_dict['prompt_ids'][train_mask])],
            data_dict['prompt_ids'][train_mask]
        )

        val_dataset = essays_dataset(
            data_dict['essay_text'][val_mask],
            data_dict['features'][val_mask],
            [normalize_scores(score, val_prompt) for score in data_dict['holistic'][val_mask]],
            data_dict['prompt_ids'][val_mask]
        )

        #dataLoaders to feed our datasets to the model in batches of 32
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)
         #initialize model and optimizer
        model = Our_BERT(hidden_size=hidden_size).to(device)
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=0.005,
            betas=(0.9, 0.999),
            weight_decay=0.1
        )
        criterion = nn.MSELoss() #minimum squared error 

        print(f"\nStarting training...")
        for epoch in range(5): 
            train_loss = train_model(model, train_loader, criterion, optimizer, epoch + 1)
            val_loss, val_qwk = evaluate_model(model, val_loader, criterion, val_prompt)

        fold_scores.append(val_qwk)
     #calculate avg and std of scores
    avg_qwk = np.mean(fold_scores)
    std_qwk = np.std(fold_scores)
    results.append({
        'hidden_size': hidden_size,
        'avg_qwk': avg_qwk,
        'std_qwk': std_qwk,
        'fold_scores': fold_scores
    })
#find the best configs
best_result = max(results, key=lambda x: x['avg_qwk'])
best_hidden_size = best_result['hidden_size']



Starting Holistic scoring using BERT model at 2024-12-07 16:12:17

Trying regression head with 0 hidden units

--------------------------------------------------
Validation prompt no. 2

Starting training...


epoch 1: avg training loss = 2.5709

--------------------Evaluating-----------------------



Validation results for prompt 2:
Avg loss: 0.0409
QWK: 0.4115
Predicted min: 1.40, max: 7.40, mean: 4.22
Actual min: 1.00, max: 6.00, mean: 3.41


epoch 2: avg training loss = 0.0359

--------------------Evaluating-----------------------



Validation results for prompt 2:
Avg loss: 0.0163
QWK: 0.6073
Predicted min: 1.18, max: 6.95, mean: 3.67
Actual min: 1.00, max: 6.00, mean: 3.41


...


Validation results for prompt 8:
Avg loss: 0.0448
QWK: 0.3292
Predicted min: 7.55, max: 67.92, mean: 46.04
Actual min: 10.00, max: 60.00, mean: 36.95


epoch 4: avg training loss = 0.0291

--------------------Evaluating-----------------------



Validation results for prompt 8:
Avg loss: 0.0928
QWK: 0.2021
Predicted min: 11.96, max: 75.95, mean: 52.53
Actual min: 10.00, max: 60.00, mean: 36.95


epoch 5: avg training loss = 0.0282

--------------------Evaluating-----------------------



Validation results for prompt 8:
Avg loss: 0.0463
QWK: 0.3346
Predicted min: 6.12, max: 68.84, mean: 45.95
Actual min: 10.00, max: 60.00, mean: 36.95


### Training final model

In [ ]:

print(f"\n{'='*50}")
print("Best option for regression head:")
print(f"Hidden units: {best_hidden_size}")
print(f"Average QWK: {best_result['avg_qwk']:.4f} (±{best_result['std_qwk']:.4f})")
print(f"{'='*50}")

#create masks for final training (prompts 2-8) and testing (prompt 1)
train_mask = np.isin(data_dict['prompt_ids'], range(2, 9))
test_mask = (data_dict['prompt_ids'] == 1)

#create final training dataset using all prompts 2-8
final_dataset = essays_dataset(
    data_dict['essay_text'][train_mask],
    data_dict['features'][train_mask],
    [normalize_scores(score, pid) for score, pid in zip(data_dict['holistic'][train_mask], data_dict['prompt_ids'][train_mask])],
    data_dict['prompt_ids'][train_mask]
)

#create test dataset using prompt 1
test_dataset = essays_dataset(
    data_dict['essay_text'][test_mask],
    data_dict['features'][test_mask],
    [normalize_scores(score, 1) for score in data_dict['holistic'][test_mask]],
    data_dict['prompt_ids'][test_mask]
)

final_loader = DataLoader(final_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

#initialize final model with best configuration
final_model = Our_BERT(hidden_size=best_hidden_size).to(device)
final_optimizer = torch.optim.AdamW(
    final_model.parameters(),
    lr=0.005,
    betas=(0.9, 0.999),
    weight_decay=0.1
)
final_criterion = nn.MSELoss()
#training final model
print("\nFinal training on prompts 2-8...")
for epoch in range(5):
    train_loss = train_model(final_model, final_loader, final_criterion, final_optimizer, epoch + 1)

print("\nEvaluating final model on Prompt 1 (test set)...")
test_loss, test_qwk = evaluate_model(final_model, test_loader, final_criterion, val_prompt_id=1)
print(f"\nFinal test results on prompt 1:")
print(f"Test loss: {test_loss:.4f}")
print(f"Test QWK: {test_qwk:.4f}")




Best option for regression head:
Hidden units: 0
Average QWK: 0.5428 (±0.1249)

Final training on prompts 2-8...


epoch 1: avg training loss = 1.4347


epoch 2: avg training loss = 0.0309


epoch 3: avg training loss = 0.0282


epoch 4: avg training loss = 0.0275


epoch 5: avg training loss = 0.0268

Evaluating final model on Prompt 1 (test set)...

--------------------Evaluating-----------------------



Validation results for prompt 1:
Avg loss: 0.0094
QWK: 0.7965
Predicted min: 2.48, max: 13.95, mean: 8.19
Actual min: 2.00, max: 12.00, mean: 8.53

Final test results on prompt 1:
Test loss: 0.0094
Test QWK: 0.7965


### Saving the model

In [ ]:
def save_model(model, optimizer, epoch, train_loss, test_qwk, path):
    #saving our model with all relavant info 
    checkpoint = { 
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'train_loss': train_loss,
        'test_qwk': test_qwk,
    }
    torch.save(checkpoint, path)

In [ ]:
path = "model-C-1"
save_model(final_model, final_optimizer, epoch + 1, train_loss, test_qwk, path)